## MNIST classification

In this notebook we tackle the perhaps most well known problem in all of machine learning, classifying hand-written digits.

The particular dataset we will use is the MNIST (Modified National Institute of Standards and Technology)
The digits are 28x28 pixel images that look somewhat like this:

![](https://user-images.githubusercontent.com/2202312/32365318-b0ccc44a-c079-11e7-8fb1-6b1566c0bdc4.png)

Each digit has been hand classified, e.g. for the above 9-7-0-9-0-...

Our task is to teach a machine to perform this classification.

# Import dependencies

This should run without errors if all dependencies are installed properly.

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
# Start a tensorflow session
session = tf.InteractiveSession()

# Set the random seed to enable reproducible code
np.random.seed(0)

# Get data and utilities

We now need to get the data we will use, which in this case is the famous [MNIST](http://yann.lecun.com/exdb/mnist/) dataset, a set of digits 70000 hand-written digits, of which 60000 are used for training and 10000 for testing.

In addition to this, we create a utility `generate_data` which generates sinograms for each digit, as well as a function `evaluate(...)` that we will use to evaluate how good the classification is.

In [3]:
# Get MNIST data
mnist = input_data.read_data_sets('MNIST_data')

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [4]:
# Read the 10000 mnist test points
batch = mnist.test.next_batch(10000)
test_images = batch[0].reshape([-1, 28, 28, 1])
test_labels = batch[1]

def evaluate(result_tensor, data_placeholder):
    """Evaluate a reconstruction method.

    Parameters
    ----------
    result_tensor : `tf.Tensor`, shape (None,)
        The tensorflow tensor containing the result of the classification.
    data_placeholder : `tf.Tensor`, shape (None, 28, 28, 1)
        The tensorflow tensor containing the input to the classification operator.

    Returns
    -------
    MSE : float
        Mean squared error of the reconstruction.
    """
    result = result_tensor.eval(
        feed_dict={data_placeholder: test_images})

    return np.mean(result == test_labels)

In [5]:
# Create placeholders. Placeholders are needed in tensorflow since tensorflow is a lazy language,
# and hence we first define the computational graph with placeholders as input, and later we evaluate it.
with tf.name_scope('placeholders'):
    images = tf.placeholder(tf.float32, shape=[None, 28, 28, 1])
    true_labels = tf.placeholder(tf.int32, shape=[None])

# Logistic regression

INSERT TEXT

In [6]:
with tf.name_scope('logistic_regression'):
    x = tf.contrib.layers.flatten(images)
    pred_one_hot = tf.contrib.layers.fully_connected(x, 10,
                                                     activation_fn=tf.nn.softmax)
    pred = tf.argmax(pred_one_hot, axis=1)
    
with tf.name_scope('optimizer'):
    one_hot_labels = tf.one_hot(true_labels, depth=10)
    
    loss = tf.reduce_mean((pred_one_hot - one_hot_labels) ** 2)
    optimizer = tf.train.AdamOptimizer().minimize(loss)

# Initialize all TF variables
session.run(tf.global_variables_initializer())

for i in range(10000):
    batch = mnist.train.next_batch(10)
    train_images = batch[0].reshape([-1, 28, 28, 1])
    train_labels = batch[1]

    session.run(optimizer, feed_dict={images: train_images, 
                                      true_labels: train_labels})

    if i % 1000 == 0:
        print('{} Average correct: {}'.format(
                i, evaluate(pred, images)))

0 Average correct: 0.117
1000 Average correct: 0.8998
2000 Average correct: 0.9156
3000 Average correct: 0.9142
4000 Average correct: 0.9168
5000 Average correct: 0.9157
6000 Average correct: 0.9218
7000 Average correct: 0.9229
8000 Average correct: 0.9229
9000 Average correct: 0.9265


# Convolutional network

INSERT TEXT

In [7]:
with tf.name_scope('convolutional_network'):
    x = tf.contrib.layers.conv2d(images, num_outputs=32, kernel_size=3, stride=2)
    x = tf.contrib.layers.conv2d(x, num_outputs=32, kernel_size=3, stride=2)
    x = tf.contrib.layers.flatten(x)
    
    pred_one_hot = tf.contrib.layers.fully_connected(x, 10,
                                                     activation_fn=tf.nn.softmax)
    pred = tf.argmax(pred_one_hot, axis=1)
    
with tf.name_scope('optimizer'):
    one_hot_labels = tf.one_hot(true_labels, depth=10)
    
    loss = tf.reduce_mean((pred_one_hot - one_hot_labels) ** 2)
    optimizer = tf.train.AdamOptimizer().minimize(loss)

# Initialize all TF variables
session.run(tf.global_variables_initializer())

for i in range(10000):
    batch = mnist.train.next_batch(10)
    train_images = batch[0].reshape([-1, 28, 28, 1])
    train_labels = batch[1]

    session.run(optimizer, feed_dict={images: train_images, 
                                      true_labels: train_labels})

    if i % 1000 == 0:
        print('{} Average correct: {}'.format(
                i, evaluate(pred, images)))

0 Average correct: 0.0998
1000 Average correct: 0.9379
2000 Average correct: 0.9562
3000 Average correct: 0.9655
4000 Average correct: 0.9665
5000 Average correct: 0.9724
6000 Average correct: 0.9679
7000 Average correct: 0.977
8000 Average correct: 0.9755
9000 Average correct: 0.9766
